# Create BRISE dataset for POTATO

In [2]:
from potato.dataset.dataset import Dataset
from potato.models.trainer import GraphTrainer

Load all data

In [121]:
import os
import json

GOLD_ATTRIBUTES = "gold_attributes"
GOLD = "labels_gold"
ANNOTATED_ATTRIBUTES = "annotated_attributes"

def load_data(dir_path, only_gold=True):
    data = []
    for filename in os.listdir(dir_path):
        with open(os.path.join(dir_path, filename), "rt") as f:
            doc = json.load(f)
            if only_gold and not doc[GOLD]:
                continue
            for sen in doc["sens"].values():
                if doc[GOLD]:
                    labels = list(set(sen[GOLD_ATTRIBUTES].keys()))
                else:
                    labels = list(set(sen[ANNOTATED_ATTRIBUTES].keys()))
                data.append({
                    "sen_id": sen["id"],
                    "text": sen["text"],
                    "labels": labels,
                })
    return data

Create dataset

In [150]:
NOT = "NOT"

TRAIN_GRAPH_GOLD_UD = "gold_train_ud.pickle"
VAL_GRAPH_GOLD_UD = "gold_val_ud.pickle"

TRAIN_GRAPH_GOLD_4LANG = "gold_train_4lang.pickle"
VAL_GRAPH_GOLD_4LANG = "gold_val_4lang.pickle"

def get_sentences(data, attribute):
    sentences = []
    for sen in data:
        if attribute in sen["labels"]:
            sentences.append((sen["text"], attribute))
        else:
            sentences.append((sen["text"], NOT))       
    return sentences

import pickle

def get_graphs(dataset, graph_path, graph_format):
    if not os.path.exists(graph_path):
        dataset.set_graphs(dataset.parse_graphs(graph_format=graph_format))
    else:
        dataset.load_graphs(graph_path)
    df = dataset.to_dataframe()
    if not os.path.exists(graph_path):
        with open(graph_path, "wb") as f:
            pickle.dump(df.graph, f)
    return df

def get_dataset(data, attribute, graph_path, save_path, graph_format):
    sentences = get_sentences(data, attribute)
    dataset = Dataset(sentences, label_vocab={NOT:0, attribute: 1})
    df = get_graphs(dataset, graph_path, graph_format)
    df.to_pickle(save_path)
    return df

Evaluate

In [99]:
from potato.graph_extractor.extract import FeatureEvaluator

def get_pred_df(df, features):
    feature_values = []
    for k in features:
        for f in features[k]:
            feature_values.append(f)
    evaluator = FeatureEvaluator()
    pred_df = evaluator.match_features(df, feature_values)
    if "label" in df:
        pred_df["label"] = df.label
    return pred_df

## Load data - only gold

In [123]:
TRAIN_DIR = "../../../../data/train"
VAL_DIR = "../../../../data/valid"

train_data = load_data(TRAIN_DIR, only_gold=True)
val_data = load_data(VAL_DIR, only_gold=True)

In [124]:
len(train_data)

1154

In [125]:
len(val_data)

100

In [126]:
train_data[10:15]

[{'sen_id': '8025_7_0',
  'text': 'Bestimmungen ohne Bezeichnung des Geltungsbereichs mit dem Planzeichen BB:',
  'labels': ['Planzeichen']},
 {'sen_id': '8025_8_0',
  'text': 'Für das gesamte Plangebiet wird bestimmt: Dächer dürfen die festgesetzte Gebäudehöhe um höchstens 4,5 m überragen.',
  'labels': ['GebaeudeHoeheArt',
   'PlangebietAllgemein',
   'AbschlussDachMaxBezugGebaeude']},
 {'sen_id': '8025_9_0',
  'text': 'Im Bauland/Wohngebiet darf die Bruttogeschoßfläche aller Geschoße, die ganz oder teilweise über dem anschließenden Gelände liegen, insgesamt höchstens 11.500 m² betragen.',
  'labels': ['WidmungUndZweckbestimmung', 'Flaechen']},
 {'sen_id': '8025_10_0',
  'text': 'Bestimmungen mit Bezeichnung des Geltungsbereichs mit dem Planzeichen BB:',
  'labels': ['Planzeichen']},
 {'sen_id': '8025_11_0',
  'text': 'Für die mit BB1 bezeichneten Grundflächen wird bestimmt: Die Unterbrechung der geschlossenen Bauweise ist zulässig',
  'labels': ['Planzeichen', 'UnterbrechungGeschlos

In [127]:
val_data[15:20]

[{'sen_id': '7080k_2_0',
  'text': 'Für die rechtliche Bedeutung der roten Planzeichen ist die beiliegende „Zeichenerklärung für den Flächenwidmungsplan und den Bebauungsplan“ (§§ 4 und 5 BO für Wien) vom 1. Oktober 2001 maßgebend, die einen Bestandteil dieses Beschlusses bildet.',
  'labels': []},
 {'sen_id': '7080k_3_0',
  'text': 'Die Bestimmungen 3.1.1 und 3.1.2.',
  'labels': []},
 {'sen_id': '7080k_3_1',
  'text': 'des Plandokumentes 7080 haben für das gesamte Plangebiet keine Gültigkeit.',
  'labels': []},
 {'sen_id': '7080k_4_0',
  'text': 'Gemäß § 5 (4) der Bauordnung für Wien wird für Teile des Plangebietes mit eigener Kennzeichnung im Plan (BB) bestimmt:',
  'labels': []},
 {'sen_id': '7080k_5_0',
  'text': 'Auf den mit BB17 bezeichneten Flächen wird die bauliche Ausnutzbarkeit mit maximal 45 % beschränkt.',
  'labels': ['Planzeichen', 'Flaechen']}]

## Planzeichen - only gold - ud

In [151]:
TRAIN_DATASET = "train_planzeichen_gold_dataset_ud"

df_train = get_dataset(train_data, "Planzeichen", TRAIN_GRAPH_GOLD_UD, TRAIN_DATASET, "ud")

In [131]:
df_train

,text,label,label_id,graph
0,MAGISTRAT DER STADT WIEN MA 21 Stadtteilplanun...,NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14,..."
1,Der Gemeinderat hat in seiner Sitzung am 29. A...,NOT,0,"(1, 3, 2, 0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,"Bezirk, Kat. Großjedlersdorf I werden unter An...",NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,Die bisherigen Flächenwidmungspläne und Bebauu...,NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 13, 10, 11, 12,..."
4,Gemäß § 4 und § 5 der BO für Wien sowie § 48 d...,NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
...,...,...,...,...
1149,Auf den mit G BB2 bezeichneten Flächen ist die...,Planzeichen,1,"(1, 2, 0, 3, 4, 5, 6, 7, 8, 9)"
1150,Innerhalb der mit BB 3 bezeichneten Flächen si...,Planzeichen,1,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 13)"
1151,Die Ausgestaltung hat zumindest durch das Aufs...,NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1152,Für die mit BB 4 bezeichneten Bereiche ist die...,Planzeichen,1,"(1, 2, 0, 3, 4, 6, 5, 7, 8, 9, 10, 11, 12, 13,..."


In [152]:
VALID_DATASET = "valid_planzeichen_gold_dataset_ud"

df_val = get_dataset(val_data, "Planzeichen", VAL_GRAPH_GOLD_UD, VALID_DATASET, "ud")

In [136]:
df_val

,text,label,label_id,graph
0,MAGISTRAT DER STADT WIEN MA 21 A Stadtteilplan...,NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11, 12, 13,..."
1,Der Gemeinderat hat in seiner Sitzung am 23. N...,NOT,0,"(1, 3, 2, 0, 4, 5, 6, 7, 8, 9, 10, 11, 13, 12,..."
2,"Bezirk, Kat. G. Leopoldstadt werden unter Anwe...",NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,Die roten Planzeichen gelten als neu festgesetzt.,NOT,0,"(1, 0, 2, 3, 4, 7, 5, 6, 8)"
4,die schwarzen Planzeichen behalten ihre Rechts...,NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 13,..."
...,...,...,...,...
95,Auf den mit BB4 bezeichneten Grundflächen ist ...,Planzeichen,1,"(1, 2, 8, 3, 4, 5, 6, 7, 0, 9, 10, 11, 12, 13)"
96,Auf der mit BB5 bezeichneten Fläche ist ein Ra...,Planzeichen,1,"(1, 0, 2, 3, 5, 4, 14, 6, 7, 8, 9, 10, 11, 12,..."
97,Auf den mit Ak öDg bezeichneten Flächen sind A...,Planzeichen,1,"(1, 4, 2, 3, 15, 5, 6, 7, 8, 11, 9, 10, 12, 13..."
98,Innerhalb des mit öDg bezeichneten Bereiches i...,Planzeichen,1,"(1, 15, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13..."


### Train and eval

In [140]:
trainer = GraphTrainer(df_train)
features = trainer.prepare_and_train()

Initializing trainer object...
Featurizing graphs by generating subgraphs up to 2...


1154it [00:15, 72.63it/s] 


Getting feature graphs...
Selecting the best features...
Generating training data...
Training...
Getting features...


In [144]:
features["Planzeichen"][:20]

[(['(u_38 / bezeichneten)'], [], 'Planzeichen'),
 (['(u_293 / Flaechen)'], [], 'Planzeichen'),
 (['(u_248 / Flaeche)'], [], 'Planzeichen'),
 (['(u_80 / PERIOD)'], [], 'Planzeichen'),
 (['(u_17 / COLON)'], [], 'Planzeichen'),
 (['(u_394 / darueber)'], [], 'Planzeichen'),
 (['(u_310 / auszubilden)'], [], 'Planzeichen'),
 (['(u_391 / Verkehrsflaeche)'], [], 'Planzeichen'),
 (['(u_26 / mit)'], [], 'Planzeichen'),
 (['(u_142 / BB)'], [], 'Planzeichen'),
 (['(u_361 / Niveau)'], [], 'Planzeichen'),
 (['(u_400 / zugeordnet)'], [], 'Planzeichen'),
 (['(u_58 / RRB)'], [], 'Planzeichen'),
 (['(u_85 / Wiener)'], [], 'Planzeichen'),
 (['(u_116 / m  :nummod (u_115 / X0))'], [], 'Planzeichen'),
 (['(u_117 / wird)'], [], 'Planzeichen'),
 (['(u_625 / Gebaeuden)'], [], 'Planzeichen'),
 (['(u_152 / SLASH)'], [], 'Planzeichen'),
 (['(u_475 / angrenzenden)'], [], 'Planzeichen'),
 (['(u_329 / Auf)'], [], 'Planzeichen')]

In [145]:
pred_df = get_pred_df(df_val, features)

100it [00:00, 226.71it/s]


In [146]:
pred_df

,Sentence,Predicted label,Matched rule,label
0,MAGISTRAT DER STADT WIEN MA 21 A Stadtteilplan...,Planzeichen,"([(u_44 / HYPHEN)], [], Planzeichen)",NOT
1,Der Gemeinderat hat in seiner Sitzung am 23. N...,Planzeichen,"([(u_38 / bezeichneten)], [], Planzeichen)",NOT
2,"Bezirk, Kat. G. Leopoldstadt werden unter Anwe...",Planzeichen,"([(u_17 / COLON)], [], Planzeichen)",NOT
3,Die roten Planzeichen gelten als neu festgesetzt.,Planzeichen,"([(u_80 / PERIOD)], [], Planzeichen)",NOT
4,die schwarzen Planzeichen behalten ihre Rechts...,Planzeichen,"([(u_80 / PERIOD)], [], Planzeichen)",NOT
...,...,...,...,...
95,Auf den mit BB4 bezeichneten Grundflächen ist ...,Planzeichen,"([(u_38 / bezeichneten)], [], Planzeichen)",Planzeichen
96,Auf der mit BB5 bezeichneten Fläche ist ein Ra...,Planzeichen,"([(u_248 / Flaeche)], [], Planzeichen)",Planzeichen
97,Auf den mit Ak öDg bezeichneten Flächen sind A...,Planzeichen,"([(u_38 / bezeichneten)], [], Planzeichen)",Planzeichen
98,Innerhalb des mit öDg bezeichneten Bereiches i...,Planzeichen,"([(u_38 / bezeichneten)], [], Planzeichen)",Planzeichen


Numerical evaluation

In [147]:
evaluator = FeatureEvaluator()

In [148]:
eval_df = evaluator.evaluate_feature("Planzeichen", features["Planzeichen"], df_val)

/home/eszter/.pyenv/versions/brise/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/eszter/.pyenv/versions/brise/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/eszter/.pyenv/versions/brise/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [149]:
eval_df[0][["Feature", "Precision", "Recall", "Fscore"]][:15]

,Feature,Precision,Recall,Fscore
0,[(u_38 / bezeichneten)],0.666667,0.333333,0.444444
1,[(u_293 / Flaechen)],0.588235,0.416667,0.487805
2,[(u_248 / Flaeche)],0.666667,0.250000,0.363636
3,[(u_80 / PERIOD)],0.338235,0.958333,0.500000
4,[(u_17 / COLON)],0.076923,0.083333,0.080000
5,[(u_394 / darueber)],0.000000,0.000000,0.000000
6,[(u_310 / auszubilden)],0.750000,0.125000,0.214286
7,[(u_391 / Verkehrsflaeche)],1.000000,0.083333,0.153846
8,[(u_26 / mit)],0.535714,0.625000,0.576923
9,[(u_142 / BB)],0.250000,0.041667,0.071429


## Planzeichen - only gold - 4lang

In [154]:
TRAIN_DATASET = "train_planzeichen_gold_dataset_4lang"

df_train = get_dataset(train_data, "Planzeichen", TRAIN_GRAPH_GOLD_4LANG, TRAIN_DATASET, "fourlang")

2021-11-18 16:57:15 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-11-18 16:57:15 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |
| depparse  | ewt     |

INFO:stanza:Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |
| depparse  | ewt     |

2021-11-18 16:57:15 INFO: Use device: cpu
INFO:stanza:Use device: cpu
2021-11-18 16:57:15 INFO: Loading: tokenize
INFO:stanza:Loading: tokenize
2021-11-18 16:57:15 INFO: Loading: pos
INFO:stanza:Loading: pos
2021-11-18 16:57:16 INFO: Loading: lemma
INFO:stanza:Loading: lemma
2021-11-18 16:57:16 INFO: Loading: depparse
INFO:stanza:Loading: depparse
2021-11-18 16:57:17 INFO: Done loading processors!
INFO:stanza:Done loading processors!
  0%|                                     